# VDEH Data Enrichment Pipeline

**Fokus:** Datenanreicherung über Deutsche Nationalbibliothek (DNB) API

## 🎯 Ziel
- Identifikation unvollständiger Datensätze
- Anreicherung fehlender Metadaten via DNB API (ISBN/ISSN)
- Konsistenzprüfung vorhandener Daten
- Validierung und Qualitätsverbesserung

## 📚 Input/Output
- **Input**: `data/vdeh/processed/03_language_detected_data.parquet`
- **Output**: `data/vdeh/processed/04_enriched_data.parquet`

## 🔗 API
- **DNB SRU API**: https://www.dnb.de/DE/Professionell/Metadatendienste/Datenbezug/SRU/sru_node.html
- **Abfrage**: ISBN/ISSN basierte Suche

In [29]:
# 🛠️ SETUP UND DATEN LADEN
import sys
from pathlib import Path

✅ Konfiguration geladen: /media/sz/Data/Bibo/analysis/config.yaml
📁 Projektroot: /media/sz/Data/Bibo/analysis
✅ Konfiguration geladen
✅ DNB API Funktionen geladen


In [30]:
# 📂 DATEN AUS VORHERIGER STUFE LADEN
processed_dir = config.project_root / config.get('paths.data.vdeh.processed')
input_path = processed_dir / '03_language_detected_data.parquet'
metadata_path = processed_dir / '03_metadata.json'

if not input_path.exists():
    raise FileNotFoundError(f"Input-Datei nicht gefunden: {input_path}\n"
                          "Bitte führen Sie zuerst 03_vdeh_language_detection.ipynb aus.")

# Daten laden
df_vdeh = pd.read_parquet(input_path)

# Vorherige Metadaten laden
with open(metadata_path, 'r') as f:
    prev_metadata = json.load(f)

print(f"📂 Daten geladen aus: {input_path}")
print(f"📊 Records: {len(df_vdeh):,}")
print(f"📋 Spalten: {list(df_vdeh.columns)}")
print(f"💾 Memory: {df_vdeh.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Quality Scores anzeigen
if 'completeness_score' in df_vdeh.columns:
    avg_completeness = df_vdeh['completeness_score'].mean()
    print(f"📊 Durchschnittliche Vollständigkeit: {avg_completeness:.1f}%")
    
    if 'quality_category' in df_vdeh.columns:
        quality_dist = df_vdeh['quality_category'].value_counts()
        print(f"📊 Qualitäts-Verteilung:")
        for cat, count in quality_dist.items():
            print(f"   {cat}: {count:,} ({count/len(df_vdeh)*100:.1f}%)")

📂 Daten geladen aus: /media/sz/Data/Bibo/analysis/data/vdeh/processed/03_language_detected_data.parquet
📊 Records: 58,760
📋 Spalten: ['id', 'title', 'authors', 'authors_affiliation', 'year', 'publisher', 'isbn', 'issn', 'authors_str', 'num_authors', 'authors_affiliation_str', 'num_authors_affiliation', 'isbn_valid', 'isbn_status', 'issn_valid', 'issn_status', 'lang_code', 'lang_confidence', 'lang_name']
💾 Memory: 49.9 MB
💾 Memory: 49.9 MB


In [31]:
# 🔍 KANDIDATEN FÜR ANREICHERUNG IDENTIFIZIEREN
print("🔍 === KANDIDATEN-IDENTIFIKATION ===\n")

# Kriterien für Anreicherungskandidaten
enrichment_candidates = pd.DataFrame()

# 1. Alle Records mit ISBN (unabhängig von Vollständigkeit)
print("📋 Kriterium 1: ISBN vorhanden (Prüfung aller Datensätze mit ISBN)")

# ISBN-basierte Kandidaten - ALLE mit ISBN
if 'isbn' in df_vdeh.columns:
    has_isbn = df_vdeh['isbn'].notna()
    
    isbn_candidates = df_vdeh[has_isbn].copy()
    
    # Statistiken für Überblick
    missing_title = isbn_candidates['title'].isna()
    missing_authors = (isbn_candidates['authors_str'].isna()) | (isbn_candidates['authors_str'] == '')
    missing_year = isbn_candidates['year'].isna()
    
    print(f"   ISBN-Kandidaten (alle): {len(isbn_candidates):,}")
    print(f"     - Vollständige Datensätze: {(~missing_title & ~missing_authors & ~missing_year).sum():,}")
    print(f"     - Unvollständig (Titel): {missing_title.sum():,}")
    print(f"     - Unvollständig (Autoren): {missing_authors.sum():,}")
    print(f"     - Unvollständig (Jahr): {missing_year.sum():,}")

# ISSN-basierte Kandidaten - nur bei unvollständigen Metadaten
if 'issn' in df_vdeh.columns:
    has_issn = df_vdeh['issn'].notna()
    missing_title = df_vdeh['title'].isna()
    missing_authors = (df_vdeh['authors_str'].isna()) | (df_vdeh['authors_str'] == '')
    missing_year = df_vdeh['year'].isna()
    
    issn_candidates = df_vdeh[
        has_issn & (missing_title | missing_authors | missing_year)
    ].copy()
    
    print(f"\n   ISSN-Kandidaten (nur unvollständig): {len(issn_candidates):,}")
    print(f"     - Fehlender Titel: {issn_candidates['title'].isna().sum():,}")
    print(f"     - Fehlende Autoren: {(issn_candidates['authors_str'].isna() | (issn_candidates['authors_str'] == '')).sum():,}")
    print(f"     - Fehlendes Jahr: {issn_candidates['year'].isna().sum():,}")

# Kombiniere Kandidaten
all_candidates = pd.concat([isbn_candidates, issn_candidates]).drop_duplicates(subset=['id'])
print(f"\n✅ Gesamt Anreicherungs-Kandidaten (mit ISBN/ISSN): {len(all_candidates):,}")

# 2. Records OHNE ISBN aber MIT Titel + Autoren (für DNB-Suche via Titel/Autor)
no_isbn_but_searchable = df_vdeh[
    (df_vdeh['isbn'].isna()) &
    (df_vdeh['title'].notna()) &
    (df_vdeh['authors_str'].notna()) &
    (df_vdeh['authors_str'] != '')
].copy()

print(f"\n📋 Kriterium 2: Ohne ISBN aber mit Titel + Autoren (DNB Titel/Autor-Suche)")
print(f"   Kandidaten: {len(no_isbn_but_searchable):,}")
print(f"     - Mit Titel: {no_isbn_but_searchable['title'].notna().sum():,}")
print(f"     - Mit Autoren: {(no_isbn_but_searchable['authors_str'].notna() & (no_isbn_but_searchable['authors_str'] != '')).sum():,}")

# Finale Kandidatenliste (unique): ISBN-Kandidaten ODER Titel+Autor-Kandidaten
final_candidates = df_vdeh[
    # ENTWEDER: ISBN vorhanden (unabhängig von Vollständigkeit)
    (df_vdeh['isbn'].notna()) |
    # ODER: Kein ISBN aber Titel + Autoren vorhanden
    (
        (df_vdeh['isbn'].isna()) &
        (df_vdeh['title'].notna()) &
        (df_vdeh['authors_str'].notna()) &
        (df_vdeh['authors_str'] != '')
    )
].copy()

print(f"\n🎯 Finale Anreicherungs-Kandidaten: {len(final_candidates):,}")
print(f"   Mit ISBN: {final_candidates['isbn'].notna().sum():,}")
print(f"   Mit ISSN: {final_candidates['issn'].notna().sum():,}")

🔍 === KANDIDATEN-IDENTIFIKATION ===

📋 Kriterium 1: ISBN vorhanden (Prüfung aller Datensätze mit ISBN)
   ISBN-Kandidaten (alle): 11,415
     - Vollständige Datensätze: 7,763
     - Unvollständig (Titel): 655
     - Unvollständig (Autoren): 3,423
     - Unvollständig (Jahr): 879

   ISSN-Kandidaten (nur unvollständig): 679
     - Fehlender Titel: 2
     - Fehlende Autoren: 667
     - Fehlendes Jahr: 650

✅ Gesamt Anreicherungs-Kandidaten (mit ISBN/ISSN): 11,415

📋 Kriterium 2: Ohne ISBN aber mit Titel + Autoren (DNB Titel/Autor-Suche)
   Kandidaten: 8,901
     - Mit Titel: 8,901
     - Mit Autoren: 8,901

🎯 Finale Anreicherungs-Kandidaten: 20,316
   Mit ISBN: 11,415
   Mit ISSN: 721


In [32]:
# 🌐 DNB API STATUS
print("🌐 === DNB API STATUS ===\n")

print("✅ DNB API Funktionen aus src/dnb_api.py geladen")
print(f"   Base URL: {DNB_SRU_BASE}")
print(f"   Schema: MARC21-xml")
print(f"   Verfügbare Funktionen:")
print(f"     - query_dnb_by_isbn(isbn, max_records=1)")
print(f"     - query_dnb_by_issn(issn, max_records=1)")
print(f"     - query_dnb_by_title_author(title, author=None, max_records=1)")

🌐 === DNB API STATUS ===

✅ DNB API Funktionen aus src/dnb_api.py geladen
   Base URL: https://services.dnb.de/sru/dnb
   Schema: MARC21-xml
   Verfügbare Funktionen:
     - query_dnb_by_isbn(isbn, max_records=1)
     - query_dnb_by_issn(issn, max_records=1)
     - query_dnb_by_title_author(title, author=None, max_records=1)


In [33]:
# 🚀 DNB DATENABFRAGE
print("🚀 === DNB DATENABFRAGE ===\n")

# Konfiguration
RATE_LIMIT_DELAY = 0.5  # Sekunden zwischen Anfragen
SAVE_INTERVAL = 50  # Speichere alle N Abfragen
DNB_DATA_FILE = processed_dir / 'dnb_raw_data.parquet'

print(f"⚙️  Konfiguration:")
print(f"   Rate Limit: {RATE_LIMIT_DELAY}s pro Anfrage")
print(f"   Save Interval: Alle {SAVE_INTERVAL} Queries")
print(f"   Output: {DNB_DATA_FILE.name}")

# Lade vorhandene DNB-Daten (falls vorhanden)
if DNB_DATA_FILE.exists():
    print(f"\n📂 Lade vorhandene DNB-Daten...")
    dnb_data_df = pd.read_parquet(DNB_DATA_FILE)
    print(f"   Bereits abgefragt: {len(dnb_data_df):,}")
    print(f"   Davon erfolgreich: {(dnb_data_df['dnb_found'] == True).sum():,}")
else:
    print(f"\n📂 Keine vorhandenen DNB-Daten gefunden - starte neue Abfrage")
    dnb_data_df = pd.DataFrame(columns=[
        'vdeh_id', 'query_type', 'query_value',
        'dnb_found', 'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher'
    ])

# Sammle ISBN/ISSN aus Kandidaten
print(f"\n📋 Extrahiere ISBN/ISSN aus {len(final_candidates):,} Kandidaten...")

queries_isbn = final_candidates[final_candidates['isbn'].notna()][['id', 'isbn']].copy()
queries_isbn.columns = ['vdeh_id', 'query_value']
queries_isbn['query_type'] = 'ISBN'

queries_issn = final_candidates[
    final_candidates['isbn'].isna() & final_candidates['issn'].notna()
][['id', 'issn']].copy()
queries_issn.columns = ['vdeh_id', 'query_value']
queries_issn['query_type'] = 'ISSN'

all_queries = pd.concat([queries_isbn, queries_issn], ignore_index=True)

print(f"   ISBN-Queries: {len(queries_isbn):,}")
print(f"   ISSN-Queries: {len(queries_issn):,}")
print(f"   Gesamt: {len(all_queries):,}")

# Filtere bereits abgefragte ISBN/ISSN
if len(dnb_data_df) > 0:
    # Erstelle Set der bereits abgefragten query_values
    already_queried = set(dnb_data_df['query_value'])
    
    # Filtere nur neue Queries
    new_queries = all_queries[~all_queries['query_value'].isin(already_queried)].copy()
    
    print(f"\n🔍 Abgleich mit vorhandenen Daten:")
    print(f"   Bereits vorhanden: {len(all_queries) - len(new_queries):,}")
    print(f"   Neu abzufragen: {len(new_queries):,}")
else:
    new_queries = all_queries
    print(f"\n🔍 Alle {len(new_queries):,} Queries sind neu")

# Nur abfragen wenn neue Queries vorhanden
if len(new_queries) > 0:
    print(f"\n🔄 Starte DNB-Abfrage für {len(new_queries):,} neue Queries...\n")
    
    from tqdm.auto import tqdm
    
    results = []
    stats = {'found': 0, 'not_found': 0}
    query_count = 0
    
    for _, row in tqdm(new_queries.iterrows(), total=len(new_queries), desc="🔍 DNB API", unit="queries"):
        # API-Abfrage
        dnb_result = None
        if row['query_type'] == 'ISBN':
            dnb_result = query_dnb_by_isbn(row['query_value'])
        elif row['query_type'] == 'ISSN':
            dnb_result = query_dnb_by_issn(row['query_value'])
        
        # Ergebnis speichern
        result_row = {
            'vdeh_id': row['vdeh_id'],
            'query_type': row['query_type'],
            'query_value': row['query_value'],
            'dnb_found': dnb_result is not None,
            'dnb_title': dnb_result.get('title') if dnb_result else None,
            'dnb_authors': ', '.join(dnb_result.get('authors', [])) if dnb_result else None,
            'dnb_year': dnb_result.get('year') if dnb_result else None,
            'dnb_publisher': dnb_result.get('publisher') if dnb_result else None
        }
        
        results.append(result_row)
        
        if dnb_result:
            stats['found'] += 1
        else:
            stats['not_found'] += 1
        
        query_count += 1
        
        # Regelmäßiges Speichern (alle SAVE_INTERVAL Queries)
        if query_count % SAVE_INTERVAL == 0:
            # Merge mit vorhandenen Daten
            new_results_df = pd.DataFrame(results)
            dnb_data_df = pd.concat([dnb_data_df, new_results_df], ignore_index=True)
            
            # Speichern
            dnb_data_df.to_parquet(DNB_DATA_FILE, index=False)
            
            # Reset results für nächste Batch
            results = []
            
            print(f"💾 Zwischenspeicherung: {query_count}/{len(new_queries)} Queries abgefragt")
        
        # Rate Limiting
        time.sleep(RATE_LIMIT_DELAY)
    
    # Finale Speicherung (restliche Ergebnisse)
    if len(results) > 0:
        new_results_df = pd.DataFrame(results)
        dnb_data_df = pd.concat([dnb_data_df, new_results_df], ignore_index=True)
        dnb_data_df.to_parquet(DNB_DATA_FILE, index=False)
    
    print(f"\n💾 DNB-Daten gespeichert: {DNB_DATA_FILE.name}")
    
    # Zusammenfassung
    print(f"\n📊 === NEUE ABFRAGEN ===")
    print(f"   Neue Queries: {len(new_queries):,}")
    print(f"   ✅ Gefunden: {stats['found']:,} ({stats['found']/len(new_queries)*100:.1f}%)")
    print(f"   ❌ Nicht gefunden: {stats['not_found']:,} ({stats['not_found']/len(new_queries)*100:.1f}%)")
    print(f"   💾 Zwischenspeicherungen: {len(new_queries)//SAVE_INTERVAL}")

else:
    print(f"\n✅ Alle ISBN/ISSN bereits in DNB-Daten vorhanden - keine neuen Abfragen nötig")

# Gesamtstatistik
print(f"\n📊 === GESAMT DNB-DATEN ===")
print(f"   Total Records: {len(dnb_data_df):,}")
print(f"   Erfolgreich: {(dnb_data_df['dnb_found'] == True).sum():,}")
print(f"   Nicht gefunden: {(dnb_data_df['dnb_found'] == False).sum():,}")

print(f"\n✅ DNB-Daten verfügbar als: dnb_data_df")
print(f"   Shape: {dnb_data_df.shape}")

🚀 === DNB DATENABFRAGE ===

⚙️  Konfiguration:
   Rate Limit: 0.5s pro Anfrage
   Save Interval: Alle 50 Queries
   Output: dnb_raw_data.parquet

📂 Lade vorhandene DNB-Daten...
   Bereits abgefragt: 11,383
   Davon erfolgreich: 6,232

📋 Extrahiere ISBN/ISSN aus 20,316 Kandidaten...
   ISBN-Queries: 11,415
   ISSN-Queries: 0
   Gesamt: 11,415

🔍 Abgleich mit vorhandenen Daten:
   Bereits vorhanden: 11,415
   Neu abzufragen: 0

✅ Alle ISBN/ISSN bereits in DNB-Daten vorhanden - keine neuen Abfragen nötig

📊 === GESAMT DNB-DATEN ===
   Total Records: 11,383
   Erfolgreich: 6,232
   Nicht gefunden: 5,151

✅ DNB-Daten verfügbar als: dnb_data_df
   Shape: (11383, 8)


In [34]:
dnb_data_df.head()

,vdeh_id,query_type,query_value,dnb_found,dnb_title,dnb_authors,dnb_year,dnb_publisher
0,aleph-publish:000000023,ISBN,3-428-05409-1,True,Die deutsche Roheisenindustrie 1871 - 1913,"Krengel, Jochen",1983.0,Duncker und Humblot
1,aleph-publish:000000038,ISBN,3-527-26070-6,True,Korrosionskunde im Experiment,"Heitz, Ewald, Henkhaus, Rolf, Rahmel, Alfred",1983.0,Verlag Chemie
2,aleph-publish:000000039,ISBN,3-802-74302-4,True,Brandschutz und Feuersicherheit in Arbeitsstä...,"Isterling, Fritz",1984.0,Vulkan-Verlag
3,aleph-publish:000000040,ISBN,3-802-70475-4,True,Lagerung von staubförmigen Schüttgütern in ...,"Koster, Karl H.",1983.0,Vulkan-Verlag Classen
4,aleph-publish:000000042,ISBN,0-853-34164-8,False,None,None,NaN,None


In [35]:
print(f"{dnb_data_df.dnb_found.sum()} von {dnb_data_df.shape[0]} Datensätze bei DNB gefunden")

6232 von 11383 Datensätze bei DNB gefunden


In [36]:
# 🔍 DNB TITEL/AUTOR-SUCHE (auch zusätzlich zu ID-Variante, für maximale Redundanz)
print("🔍 === DNB TITEL/AUTOR-SUCHE ===\n")

# Konfiguration
DNB_TITLE_DATA_FILE = processed_dir / 'dnb_title_author_data.parquet'
ALWAYS_TA_FOR_ALL_WITH_TITLE_AUTHORS = True  # Variante A: maximale Redundanz

print(f"⚙️  Konfiguration:")
print(f"   Rate Limit: {RATE_LIMIT_DELAY}s pro Anfrage")
print(f"   Save Interval: Alle {SAVE_INTERVAL} Queries")
print(f"   Output: {DNB_TITLE_DATA_FILE.name}")
print(f"   TA für alle Titel+Autoren: {ALWAYS_TA_FOR_ALL_WITH_TITLE_AUTHORS}")

# Lade vorhandene Titel/Autor-Suchdaten (falls vorhanden)
if DNB_TITLE_DATA_FILE.exists():
    print(f"\n📂 Lade vorhandene Titel/Autor-Suchdaten...")
    dnb_title_df = pd.read_parquet(DNB_TITLE_DATA_FILE)
    print(f"   Bereits abgefragt: {len(dnb_title_df):,}")
    print(f"   Davon erfolgreich: {(dnb_title_df['dnb_found'] == True).sum():,}")
else:
    print(f"\n📂 Keine vorhandenen Titel/Autor-Suchdaten gefunden - starte neue Abfrage")
    dnb_title_df = pd.DataFrame(columns=[
        'vdeh_id', 'query_type', 'title', 'author',
        'dnb_found', 'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher'
    ])

# Identifiziere Kandidaten für Titel/Autor-Suche
if ALWAYS_TA_FOR_ALL_WITH_TITLE_AUTHORS:
    title_author_candidates = df_vdeh[
        (df_vdeh['title'].notna()) &
        (df_vdeh['authors_str'].notna()) &
        (df_vdeh['authors_str'] != '')
    ].copy()
else:
    title_author_candidates = df_vdeh[
        (df_vdeh['isbn'].isna()) &
        (df_vdeh['issn'].isna()) &
        (df_vdeh['title'].notna()) &
        (df_vdeh['authors_str'].notna()) &
        (df_vdeh['authors_str'] != '')
    ].copy()

print(f"\n📋 Titel/Autor-Kandidaten: {len(title_author_candidates):,}")
print(f"   Mit Titel: {title_author_candidates['title'].notna().sum():,}")
print(f"   Mit Autoren: {(title_author_candidates['authors_str'].notna() & (title_author_candidates['authors_str'] != '')).sum():,}")

# Erstelle Query-Liste
title_queries = title_author_candidates[['id', 'title', 'authors_str']].copy()
title_queries.columns = ['vdeh_id', 'title', 'author']
title_queries['query_type'] = 'TITLE_AUTHOR'

print(f"   Gesamt Titel/Autor-Queries (vor Deduplikation): {len(title_queries):,}")

# Filtere bereits abgefragte Titel/Autor-Kombinationen
if len(dnb_title_df) > 0:
    # Erstelle Set der bereits abgefragten vdeh_ids
    already_queried = set(dnb_title_df['vdeh_id'])
    
    # Filtere nur neue Queries
    new_title_queries = title_queries[~title_queries['vdeh_id'].isin(already_queried)].copy()
    
    print(f"\n🔍 Abgleich mit vorhandenen Daten:")
    print(f"   Bereits vorhanden: {len(title_queries) - len(new_title_queries):,}")
    print(f"   Neu abzufragen: {len(new_title_queries):,}")
else:
    new_title_queries = title_queries
    print(f"\n🔍 Alle {len(new_title_queries):,} Titel/Autor-Queries sind neu")

# Nur abfragen wenn neue Queries vorhanden
if len(new_title_queries) > 0:
    print(f"\n🔄 Starte DNB Titel/Autor-Abfrage für {len(new_title_queries):,} neue Queries...\n")
    
    from tqdm.auto import tqdm
    
    results = []
    stats = {'found': 0, 'not_found': 0}
    query_count = 0
    
    for _, row in tqdm(new_title_queries.iterrows(), total=len(new_title_queries), desc="🔍 DNB Titel/Autor", unit="queries"):
        # API-Abfrage
        dnb_result = query_dnb_by_title_author(row['title'], row['author'])
        
        # Ergebnis speichern
        result_row = {
            'vdeh_id': row['vdeh_id'],
            'query_type': row['query_type'],
            'title': row['title'],
            'author': row['author'],
            'dnb_found': dnb_result is not None,
            'dnb_title': dnb_result.get('title') if dnb_result else None,
            'dnb_authors': ', '.join(dnb_result.get('authors', [])) if dnb_result else None,
            'dnb_year': dnb_result.get('year') if dnb_result else None,
            'dnb_publisher': dnb_result.get('publisher') if dnb_result else None
        }
        
        results.append(result_row)
        
        if dnb_result:
            stats['found'] += 1
        else:
            stats['not_found'] += 1
        
        query_count += 1
        
        # Regelmäßiges Speichern (alle SAVE_INTERVAL Queries)
        if query_count % SAVE_INTERVAL == 0:
            # Merge mit vorhandenen Daten
            new_results_df = pd.DataFrame(results)
            dnb_title_df = pd.concat([dnb_title_df, new_results_df], ignore_index=True)
            
            # Speichern
            dnb_title_df.to_parquet(DNB_TITLE_DATA_FILE, index=False)
            
            # Reset results für nächste Batch
            results = []
            
            print(f"💾 Zwischenspeicherung: {query_count}/{len(new_title_queries)} Queries abgefragt")
        
        # Rate Limiting
        time.sleep(RATE_LIMIT_DELAY)
    
    # Finale Speicherung (restliche Ergebnisse)
    if len(results) > 0:
        new_results_df = pd.DataFrame(results)
        dnb_title_df = pd.concat([dnb_title_df, new_results_df], ignore_index=True)
        dnb_title_df.to_parquet(DNB_TITLE_DATA_FILE, index=False)
    
    print(f"\n💾 DNB Titel/Autor-Daten gespeichert: {DNB_TITLE_DATA_FILE.name}")
    
    # Zusammenfassung
    print(f"\n📊 === NEUE TITEL/AUTOR-ABFRAGEN ===")
    print(f"   Neue Queries: {len(new_title_queries):,}")
    print(f"   ✅ Gefunden: {stats['found']:,} ({stats['found']/len(new_title_queries)*100:.1f}%)")
    print(f"   ❌ Nicht gefunden: {stats['not_found']:,} ({stats['not_found']/len(new_title_queries)*100:.1f}%)")
    print(f"   💾 Zwischenspeicherungen: {len(new_title_queries)//SAVE_INTERVAL}")

else:
    print(f"\n✅ Alle Titel/Autor-Kombinationen bereits abgefragt - keine neuen Abfragen nötig")

# Gesamtstatistik
print(f"\n📊 === GESAMT TITEL/AUTOR-DATEN ===")
print(f"   Total Records: {len(dnb_title_df):,}")
print(f"   Erfolgreich: {(dnb_title_df['dnb_found'] == True).sum():,}")
print(f"   Nicht gefunden: {(dnb_title_df['dnb_found'] == False).sum():,}")

print(f"\n✅ Titel/Autor-Daten verfügbar als: dnb_title_df")
print(f"   Shape: {dnb_title_df.shape}")

🔍 === DNB TITEL/AUTOR-SUCHE ===

⚙️  Konfiguration:
   Rate Limit: 0.5s pro Anfrage
   Save Interval: Alle 50 Queries
   Output: dnb_title_author_data.parquet
   TA für alle Titel+Autoren: True

📂 Lade vorhandene Titel/Autor-Suchdaten...
   Bereits abgefragt: 8,901
   Davon erfolgreich: 2,724



📋 Titel/Autor-Kandidaten: 16,804
   Mit Titel: 16,804
   Mit Autoren: 16,804
   Gesamt Titel/Autor-Queries (vor Deduplikation): 16,804

🔍 Abgleich mit vorhandenen Daten:
   Bereits vorhanden: 8,901
   Neu abzufragen: 7,903

🔄 Starte DNB Titel/Autor-Abfrage für 7,903 neue Queries...



🔍 DNB Titel/Autor:   0%|          | 0/7903 [00:00<?, ?queries/s]

💾 Zwischenspeicherung: 50/7903 Queries abgefragt
💾 Zwischenspeicherung: 100/7903 Queries abgefragt
💾 Zwischenspeicherung: 100/7903 Queries abgefragt
💾 Zwischenspeicherung: 150/7903 Queries abgefragt
💾 Zwischenspeicherung: 150/7903 Queries abgefragt
💾 Zwischenspeicherung: 200/7903 Queries abgefragt
💾 Zwischenspeicherung: 200/7903 Queries abgefragt
💾 Zwischenspeicherung: 250/7903 Queries abgefragt
💾 Zwischenspeicherung: 250/7903 Queries abgefragt
💾 Zwischenspeicherung: 300/7903 Queries abgefragt
💾 Zwischenspeicherung: 300/7903 Queries abgefragt
💾 Zwischenspeicherung: 350/7903 Queries abgefragt
💾 Zwischenspeicherung: 350/7903 Queries abgefragt
💾 Zwischenspeicherung: 400/7903 Queries abgefragt
💾 Zwischenspeicherung: 400/7903 Queries abgefragt
💾 Zwischenspeicherung: 450/7903 Queries abgefragt
💾 Zwischenspeicherung: 450/7903 Queries abgefragt
💾 Zwischenspeicherung: 500/7903 Queries abgefragt
💾 Zwischenspeicherung: 500/7903 Queries abgefragt
💾 Zwischenspeicherung: 550/7903 Queries abgefragt
💾

/tmp/ipykernel_953440/1790920450.py:109: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dnb_title_df = pd.concat([dnb_title_df, new_results_df], ignore_index=True)


💾 Zwischenspeicherung: 2100/7903 Queries abgefragt
💾 Zwischenspeicherung: 2150/7903 Queries abgefragt
💾 Zwischenspeicherung: 2150/7903 Queries abgefragt
💾 Zwischenspeicherung: 2200/7903 Queries abgefragt
💾 Zwischenspeicherung: 2200/7903 Queries abgefragt
💾 Zwischenspeicherung: 2250/7903 Queries abgefragt
💾 Zwischenspeicherung: 2250/7903 Queries abgefragt
💾 Zwischenspeicherung: 2300/7903 Queries abgefragt
💾 Zwischenspeicherung: 2300/7903 Queries abgefragt
💾 Zwischenspeicherung: 2350/7903 Queries abgefragt
💾 Zwischenspeicherung: 2350/7903 Queries abgefragt
💾 Zwischenspeicherung: 2400/7903 Queries abgefragt
💾 Zwischenspeicherung: 2400/7903 Queries abgefragt
💾 Zwischenspeicherung: 2450/7903 Queries abgefragt
💾 Zwischenspeicherung: 2450/7903 Queries abgefragt
💾 Zwischenspeicherung: 2500/7903 Queries abgefragt
💾 Zwischenspeicherung: 2500/7903 Queries abgefragt
💾 Zwischenspeicherung: 2550/7903 Queries abgefragt
💾 Zwischenspeicherung: 2550/7903 Queries abgefragt
💾 Zwischenspeicherung: 2600/790

In [37]:
dnb_title_df.dnb_found.sum()

5729

In [38]:
# 🔗 DNB-DATEN MIT VDEH-DATEN ZUSAMMENFÜHREN
print("🔗 === DNB-DATEN MERGE ===\n")

# Starte mit VDEH-Daten
df_enriched = df_vdeh.copy()

# 1. Merge ISBN/ISSN-basierte DNB-Daten (als ID-Variante)
if len(dnb_data_df) > 0:
    dnb_isbn_issn = dnb_data_df[dnb_data_df['dnb_found'] == True][
        ['vdeh_id', 'query_type', 'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher']
    ].rename(columns={'query_type': 'dnb_query_method'})
    
    df_enriched = df_enriched.merge(
        dnb_isbn_issn,
        left_on='id',
        right_on='vdeh_id',
        how='left',
        suffixes=('', '_dup')
    )
    if 'vdeh_id' in df_enriched.columns:
        df_enriched.drop(columns=['vdeh_id'], inplace=True)
    if 'dnb_title_dup' in df_enriched.columns:
        df_enriched.drop(columns=[c for c in df_enriched.columns if c.endswith('_dup')], inplace=True)
    print(f"✅ ISBN/ISSN-basierte DNB-Daten (ID) gemerged → Spalten: dnb_title, dnb_authors, dnb_year, dnb_publisher")
    print(f"   ID-Matches: {df_enriched['dnb_query_method'].notna().sum():,}")

# 2. Merge Titel/Autor-basierte DNB-Daten als separate Variante (_ta)
if len(dnb_title_df) > 0:
    dnb_title_matches = dnb_title_df[dnb_title_df['dnb_found'] == True][
        ['vdeh_id', 'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher']
    ].copy()
    dnb_title_matches = dnb_title_matches.rename(columns={
        'dnb_title': 'dnb_title_ta',
        'dnb_authors': 'dnb_authors_ta',
        'dnb_year': 'dnb_year_ta',
        'dnb_publisher': 'dnb_publisher_ta'
    })
    df_enriched = df_enriched.merge(
        dnb_title_matches,
        left_on='id',
        right_on='vdeh_id',
        how='left'
    )
    if 'vdeh_id' in df_enriched.columns:
        df_enriched.drop(columns=['vdeh_id'], inplace=True)
    print(f"✅ Titel/Autor-basierte DNB-Daten (TA) gemerged → Spalten: dnb_*_ta")
    print(f"   TA-Matches: {df_enriched[['dnb_title_ta','dnb_authors_ta','dnb_year_ta','dnb_publisher_ta']].notna().any(axis=1).sum():,}")

# 3. Rückwärtskompatibilität: dnb_query_method belassen (zeigt ID vs Titel/Autor als Primärquelle)
#    (wird künftig nur noch als Hinweis genutzt; Fusion vergleicht explizit beide Varianten)

# Zusammenfassung
print(f"\n📊 === MERGE ZUSAMMENFASSUNG ===")
print(f"   Total Records: {len(df_enriched):,}")
print(f"   Mit ID-DNB: {df_enriched['dnb_query_method'].notna().sum():,}")
print(f"   Mit TA-DNB: {df_enriched[['dnb_title_ta','dnb_authors_ta','dnb_year_ta','dnb_publisher_ta']].notna().any(axis=1).sum():,}")

print(f"\n✅ df_enriched erstellt und bereit zum Speichern")


🔗 === DNB-DATEN MERGE ===

✅ ISBN/ISSN-basierte DNB-Daten (ID) gemerged → Spalten: dnb_title, dnb_authors, dnb_year, dnb_publisher
   ID-Matches: 6,232
✅ Titel/Autor-basierte DNB-Daten (TA) gemerged → Spalten: dnb_*_ta
   TA-Matches: 5,729

📊 === MERGE ZUSAMMENFASSUNG ===
   Total Records: 58,760
   Mit ID-DNB: 6,232
   Mit TA-DNB: 5,729

✅ df_enriched erstellt und bereit zum Speichern
   Mit TA-DNB: 5,729

✅ df_enriched erstellt und bereit zum Speichern


In [39]:
# 🔧 DATENTYP-NORMALISIERUNG (VOR dem Speichern!)
print("🔧 === DATENTYP-NORMALISIERUNG ===\n")

# Konvertiere Jahr-Spalten zu Int64 (verhindert float/int Konflikte bei der Fusion)
year_columns = ['year', 'dnb_year', 'dnb_year_ta']

for col in year_columns:
    if col in df_enriched.columns:
        # Konvertiere zu Int64, NaN bleiben NaN
        original_count = df_enriched[col].notna().sum()
        df_enriched[col] = pd.to_numeric(df_enriched[col], errors='coerce').astype('Int64')
        new_count = df_enriched[col].notna().sum()
        
        print(f"   {col}: {original_count:,} → {new_count:,} (Int64)")
        
        if original_count != new_count:
            print(f"      ⚠️  {original_count - new_count:,} Werte konnten nicht konvertiert werden")

print(f"\n✅ Datentypen normalisiert - bereit zum Speichern")


🔧 === DATENTYP-NORMALISIERUNG ===

   year: 33,687 → 33,687 (Int64)
   dnb_year: 6,199 → 6,199 (Int64)
   dnb_year_ta: 4,445 → 4,445 (Int64)

✅ Datentypen normalisiert - bereit zum Speichern


In [40]:
# 💾 DATEN SPEICHERN (vor KI-Fusion)
print("💾 === DATEN SPEICHERN ===\n")

# Output-Pfade
output_path = processed_dir / '04_dnb_enriched_data.parquet'
metadata_output = processed_dir / '04_metadata.json'

# 1. Parquet speichern
df_enriched.to_parquet(output_path, index=False)
print(f"✅ DNB-angereicherte Daten gespeichert: {output_path.name}")
print(f"   Records: {len(df_enriched):,}")
print(f"   Spalten: {len(df_enriched.columns)}")
print(f"   Größe: {output_path.stat().st_size / 1024**2:.1f} MB")

# 2. Metadaten erstellen
metadata = {
    'step': '04_dnb_enrichment',
    'input_file': '03_language_detected_data.parquet',
    'output_file': '04_dnb_enriched_data.parquet',
    'timestamp': pd.Timestamp.now().isoformat(),
    'record_count': len(df_enriched),
    'columns': list(df_enriched.columns),
    
    # DNB Query Statistiken
    'dnb_queries': {
        'isbn_issn': {
            'total_queries': len(dnb_data_df) if len(dnb_data_df) > 0 else 0,
            'successful': int((dnb_data_df['dnb_found'] == True).sum()) if len(dnb_data_df) > 0 else 0,
            'failed': int((dnb_data_df['dnb_found'] == False).sum()) if len(dnb_data_df) > 0 else 0
        },
        'title_author': {
            'total_queries': len(dnb_title_df) if len(dnb_title_df) > 0 else 0,
            'successful': int((dnb_title_df['dnb_found'] == True).sum()) if len(dnb_title_df) > 0 else 0,
            'failed': int((dnb_title_df['dnb_found'] == False).sum()) if len(dnb_title_df) > 0 else 0
        }
    },
    
    # DNB-Daten Verfügbarkeit (Varianten)
    'dnb_variants': {
        'id_available': int(df_enriched['dnb_query_method'].notna().sum()) if 'dnb_query_method' in df_enriched.columns else 0,
        'ta_available': int(df_enriched[['dnb_title_ta','dnb_authors_ta','dnb_year_ta','dnb_publisher_ta']].notna().any(axis=1).sum())
    },
    
    # DNB-Feldverfügbarkeit
    'dnb_field_availability': {
        'id': {
            'title': int(df_enriched['dnb_title'].notna().sum()) if 'dnb_title' in df_enriched.columns else 0,
            'authors': int(df_enriched['dnb_authors'].notna().sum()) if 'dnb_authors' in df_enriched.columns else 0,
            'year': int(df_enriched['dnb_year'].notna().sum()) if 'dnb_year' in df_enriched.columns else 0,
            'publisher': int(df_enriched['dnb_publisher'].notna().sum()) if 'dnb_publisher' in df_enriched.columns else 0
        },
        'title_author': {
            'title': int(df_enriched['dnb_title_ta'].notna().sum()) if 'dnb_title_ta' in df_enriched.columns else 0,
            'authors': int(df_enriched['dnb_authors_ta'].notna().sum()) if 'dnb_authors_ta' in df_enriched.columns else 0,
            'year': int(df_enriched['dnb_year_ta'].notna().sum()) if 'dnb_year_ta' in df_enriched.columns else 0,
            'publisher': int(df_enriched['dnb_publisher_ta'].notna().sum()) if 'dnb_publisher_ta' in df_enriched.columns else 0
        }
    },
    
    # Originaldaten-Vollständigkeit
    'vdeh_completeness': {
        'title': int(df_enriched['title'].notna().sum()),
        'authors': int((df_enriched['authors_str'].notna() & (df_enriched['authors_str'] != '')).sum()),
        'year': int(df_enriched['year'].notna().sum()),
        'publisher': int(df_enriched['publisher'].notna().sum())
    }
}

# Metadaten speichern
with open(metadata_output, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"\n✅ Metadaten gespeichert: {metadata_output.name}")

# 3. Zusammenfassung
print(f"\n📊 === DNB ENRICHMENT ABGESCHLOSSEN ===")
print(f"   Input: {len(df_vdeh):,} VDEH Records")
print(f"   Output: {len(df_enriched):,} Records mit DNB-Daten")
print(f"   DNB-Queries: {metadata['dnb_queries']['isbn_issn']['total_queries'] + metadata['dnb_queries']['title_author']['total_queries']:,}")
print(f"   ID-Variante verfügbar: {metadata['dnb_variants']['id_available']:,}")
print(f"   TA-Variante verfügbar: {metadata['dnb_variants']['ta_available']:,}")

print(f"\n➡️  Nächster Schritt: 05_vdeh_data_fusion.ipynb")
print(f"   KI-gestützte Fusion von VDEH und DNB Daten (beide Varianten)")

print(f"\n🎉 DNB Enrichment erfolgreich abgeschlossen!")

💾 === DATEN SPEICHERN ===

✅ DNB-angereicherte Daten gespeichert: 04_dnb_enriched_data.parquet
   Records: 58,760
   Spalten: 28
   Größe: 4.3 MB

✅ Metadaten gespeichert: 04_metadata.json

📊 === DNB ENRICHMENT ABGESCHLOSSEN ===
   Input: 58,760 VDEH Records
   Output: 58,760 Records mit DNB-Daten
   DNB-Queries: 28,187
   ID-Variante verfügbar: 6,232
   TA-Variante verfügbar: 5,729

➡️  Nächster Schritt: 05_vdeh_data_fusion.ipynb
   KI-gestützte Fusion von VDEH und DNB Daten (beide Varianten)

🎉 DNB Enrichment erfolgreich abgeschlossen!

✅ Metadaten gespeichert: 04_metadata.json

📊 === DNB ENRICHMENT ABGESCHLOSSEN ===
   Input: 58,760 VDEH Records
   Output: 58,760 Records mit DNB-Daten
   DNB-Queries: 28,187
   ID-Variante verfügbar: 6,232
   TA-Variante verfügbar: 5,729

➡️  Nächster Schritt: 05_vdeh_data_fusion.ipynb
   KI-gestützte Fusion von VDEH und DNB Daten (beide Varianten)

🎉 DNB Enrichment erfolgreich abgeschlossen!
